# Как достать признаки с LizaAlert и оформить это в jupyter

Сначала импортируем необходимые пакеты из соответствующих библиотек

In [64]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import scrapy
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikzo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [65]:
my_url = 'https://lizaalert.org/forum/viewtopic.php?f=241&t=12498'
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=120&t=2360"
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25112"# boy and AK-47
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25087" #девочка с необычным лицом
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25197" #косоглазый чел
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25104" #ded anisimov
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25098" #ded Medvedev
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25109" #дед Павлов
#my_url = 'https://lizaalert.org/forum/viewtopic.php?f=276&t=25083'#boy Sayakbaev
#my_url = 'https://lizaalert.org/forum/viewtopic.php?f=276&t=25078' #grandma Leandrova
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25077" #girl Kamilla
client = uReq(my_url)  # создаем документ, являющийся сырым кодом сайта

In [66]:
page_html = client.read() # функция .read() позволяет вывести на экран все содержимое файла, а если мы это сохраним в переменную, 
                          # то получим объект, который хранит в себе все строки этого файла

In [67]:
client.close()

In [68]:
ps = soup(page_html, "html.parser")

In [69]:
type(ps)

bs4.BeautifulSoup

In [70]:
english_stops = ['.', ',', ':', ';', "!", "?", "-"]

Далее, для работы понадобится открыть код страницы, которую мы хотим парсить. Для этого правой кнопки мыши нажимаем на страницу, затем выбираем просмотреть код, который затем должен появиться в правой стороне страницы.

In [71]:
containers = ps.find_all("div", {"class": "content"})
len(containers)

3

In [72]:
main = ps.find('div', {'class':'content'}).text

In [73]:
posts = ps.find('div', {'class':'pagination'})
posts
re.search(r'\d+', str(posts)).group()

'3'

In [74]:
re.search(r'\d+', str(posts)).group()

'3'

Код для возраста 

In [75]:
title = ps.find("title").extract()
title

<title>Жива. Пахатаева Дарья, 2009 г.р. г. Караганда • Форум lizaalert.org</title>

In [76]:
re.search(r'\d+', str(title)).group()

'2009'

In [77]:
container = containers[0]
containers[0]

<div class="content">Внимание! Пропал ребенок! <br/>4 июля 2016 в районе шахты 33 пропала Дарья Пахатаева, 2009 года рождения. <br/><br/><span style="font-weight: bold">Приметы:</span> Рост: 110-115 см., волосы светлые, короткая стрижка, серые глаза.<br/><br/><span style="font-weight: bold">Была одета:</span> Синие джинсы, сандали вишневого цвета.<br/><br/><div class="inline-attachment">
<dl class="file">
<dt class="attach-image"><img alt="ZXnw5gefOY4.jpg" onclick="viewableArea(this);" src="./download/file.php?id=6335&amp;sid=36bce3b9b9993ffce6e9ecd27bd829eb"/></dt>
<dd>ZXnw5gefOY4.jpg (104.81 Кб) Просмотров: 1218</dd>
</dl>
</div><br/><br/><span style="font-weight: bold"><a class="postlink" href="https://yadi.sk/i/ZONnlSq7t6zMx/" rel="nofollow" target="_blank">Ориентировка на печать</a></span></div>

In [78]:
container.img["alt"]  # проверили наличие или отсутствие изображения

'ZXnw5gefOY4.jpg'

In [79]:
contents = container.contents
contents

['Внимание! Пропал ребенок! ',
 <br/>,
 '4 июля 2016 в районе шахты 33 пропала Дарья Пахатаева, 2009 года рождения. ',
 <br/>,
 <br/>,
 <span style="font-weight: bold">Приметы:</span>,
 ' Рост: 110-115 см., волосы светлые, короткая стрижка, серые глаза.',
 <br/>,
 <br/>,
 <span style="font-weight: bold">Была одета:</span>,
 ' Синие джинсы, сандали вишневого цвета.',
 <br/>,
 <br/>,
 <div class="inline-attachment">
 <dl class="file">
 <dt class="attach-image"><img alt="ZXnw5gefOY4.jpg" onclick="viewableArea(this);" src="./download/file.php?id=6335&amp;sid=36bce3b9b9993ffce6e9ecd27bd829eb"/></dt>
 <dd>ZXnw5gefOY4.jpg (104.81 Кб) Просмотров: 1218</dd>
 </dl>
 </div>,
 <br/>,
 <br/>,
 <span style="font-weight: bold"><a class="postlink" href="https://yadi.sk/i/ZONnlSq7t6zMx/" rel="nofollow" target="_blank">Ориентировка на печать</a></span>]

Код, чтобы достать телесложение

In [80]:
for string in contents:
    if "елосложени" in string:
        no_stops_string = ''.join([t for t in string if t not in english_stops])
        tokens = word_tokenize(no_stops_string)
for i in range(len(tokens)):
    if "елосложени" in tokens[i]:
        if tokens[i][-1] == "я":
            body = tokens[i - 1]
        else:
            body = tokens[i + 1]
body.lower()
a = WordNetLemmatizer()
a.lemmatize(body)

'-'

In [81]:
body

'-'

In [90]:
## Нужно доработать код! Или сделать заново
def eyes_color(contents):
    ec = "Не указано"
    for string in contents:
        if "лаз" in string:
            no_stops_string = ''.join([t for t in string if t not in english_stops])
            tokens = word_tokenize(no_stops_string)
            tokens.append("Кто-то объебался")
            for i in range(len(tokens)):
                if "лаз" in tokens[i]:
                    ec = tokens[i + 1]
                if ec == "Кто-то объебался":
                    ec = tokens[i-1]
                    break
    return ec
eyes_color(contents)

'серые'

Код, чтобы достать цвет волос

In [83]:

for string in contents:
    if "олосы" in string:
        no_stops_string_1 = ''.join([t for t in string if t not in english_stops])
        tokens_1 = word_tokenize(no_stops_string_1)
for i in range(len(tokens_1)):
    if "олосы" in tokens_1[i]:
        hair_colour = tokens_1[i + 1]
hair_colour.lower()
b = WordNetLemmatizer()
b.lemmatize(hair_colour)

'светлые'

In [84]:
def hair_color(contents):
    hc = ''
    for string in contents:
        if "олосы" in string:
            no_stops_string_1 = ''.join([t for t in string if t not in english_stops])
            tokens_1 = word_tokenize(no_stops_string_1)
            for i in range(len(tokens_1)):
                if "олосы" in tokens_1[i]:
                    hc = tokens_1[i + 1]
                    break
    return hc

In [85]:
hair_color(contents)

'светлые'

In [86]:
tokens_1

['Рост',
 '110115',
 'см',
 'волосы',
 'светлые',
 'короткая',
 'стрижка',
 'серые',
 'глаза']

Сделаем вспомогательные словари для следующего шага

In [87]:
months_length = {"янв": 31, "фев": 28, "мар": 31, "апр": 30, "май": 31, "июн": 30, "июл": 31, "авг": 31, "сен": "30",
                "окт": 31, "ноя": 30, "дек": 31}

In [88]:
loss_find = {"время пропажи": "", "время нахождения": ""}

Код, чтобы достать время пропажи

In [89]:
for string in contents:
    if "2019" in string:
        tokeny = word_tokenize(string)
        break
lost_time = tokeny[tokeny.index("2019") - 2] + " " + tokeny[tokeny.index("2019") - 1] # здесь нужно придумать, как распространить это на все года
    

NameError: name 'tokeny' is not defined

In [ ]:
lost_time = lost_time.replace(re.search(r' \w+', lost_time).group(), re.search(r' \w{3}', lost_time).group())
lost_time

In [ ]:
loss_find["время пропажи"] = lost_time

Код, чтобы достать время нахождения

In [ ]:
postbody = ps.find('div', {"class":"notice"}).text

In [ ]:
postbody

In [ ]:
find_time = re.search(r'\d{2} \w+', str(postbody)).group()
find_time

In [ ]:
if "0" in find_time:
    find_time = find_time.replace("0", "")

In [ ]:
loss_find["время нахождения"] = find_time

In [ ]:
loss_find

# А теперь реализуем то, ради чего были найдены предыдущие два пункта: время поиска

In [ ]:
ml = list(months_length.keys())
lt = re.search(r'\w{3}', loss_find["время пропажи"]).group()
ft = re.search(r'\w{3}', loss_find["время нахождения"]).group()

In [ ]:
if lt == ft:
    search_time = int(re.search(r'\d+', loss_find["время нахождения"]).group()) - int(re,search(r'\d+', loss_find["время пропажи"]).group())
elif ml.index(ft) > ml.index(lt):
    search_time = int(re.search(r'\d+', loss_find["время нахождения"]).group())
    for i in range(ml.index(ft) - 1, ml.index(lt) - 1, -1):
        if ml[i] != lt:
            search_time += months_length[ml[i]]
        else:
            search_time += months_length[ml[i]] - int(re.search(r'\d+', loss_find["время пропажи"]).group())
elif ml.index(ft) < ml.index(lt):
    search_time = int(re.search(r'\d+', loss_find["время нахождения"]).group())
    i = ml.index(ft) - 1
    while i >= 0:
        if ml[i] != lt:
            search_time += months_length[ml[i]]
        i -= 1
    for j in range(len(ml) - 1, ml.index(lt) - 1, -1):
        if ml[j] != lt:
            search_time += months_length[ml[j]]
        else:
            search_time += months_length[ml[j]] - int(re.search(r'\d+', loss_find["время пропажи"]).group())
        
    

            
    
        
        

In [ ]:
search_time

In [ ]:
containers2 = ps.findAll("div", {"class":"content"})[0].contents
containers2

In [ ]:
containers.contents

Код для доставания возраста 